In [2]:
from pathlib import Path
import sys
import os
import django
import pandas as pd
from asgiref.sync import sync_to_async

# Add the parent directory to the system path and setup django
BASE_DIR = str(Path(os.getcwd()).resolve().parent.parent.parent.parent)

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

file_name = "LibrarySupportProjections_OAPEN_DOAB_20231127_SD.xlsx"
file_input_folder = (
    "/home/guillaume-alzieu/Dev/TSOSI/TSOSI_data/doab/0_to_combine/"
)
file_output_folder = "/home/guillaume-alzieu/Dev/TSOSI/TSOSI_data/doab/1_raw/"


def pre_process():
    sheet_data = "Details"
    sheet_raw = "Raw Data"
    name_column = "Institution"
    country_colum = "Country"

    xls = pd.ExcelFile(f"{file_input_folder}{file_name}")
    df_data = pd.read_excel(xls, sheet_data)
    df_raw = pd.read_excel(xls, sheet_raw)
    country_mapping = df_raw.dropna(subset=country_colum).drop_duplicates(
        subset=[name_column, country_colum]
    )
    # de-duplicate
    assert len(country_mapping) == len(
        country_mapping.drop_duplicates(subset=[name_column])
    )
    country_mapping = country_mapping.set_index(name_column)[country_colum]
    df_data[country_colum] = df_data[name_column].map(country_mapping)
    columns = df_data.columns
    ordered_columns = []
    for col in columns:
        if col != country_colum:
            ordered_columns.append(col)
        if col == name_column:
            ordered_columns.append(country_colum)

    date_cols = [
        "Support Confirmation Date",
        "Support Start Date",
        "Support End Date",
    ]
    for d_col in date_cols:
        df_data[d_col] = df_data[d_col].dt.date
    return df_data[ordered_columns].copy()


res = pre_process()

In [57]:
file_path = "2023-11-27-DOAB_Library_Report_2023.xlsx"
res.to_excel(f"{file_output_folder}{file_path}", index=False)